In [200]:
import glob
import pandas as pd
from collections import Counter

In [201]:
#Read artists master data that will be joined to concatenated list of all social data
artists = pd.read_csv('master_artists_list.csv')

In [202]:
artists.columns

Index(['artist_name', 'age', 'years_active', 'genre_pop', 'genre_rock',
       'genre_hip_hop', 'genre_world_music', 'genre_r_and_b',
       'genre_spiritual', 'genre_electronic', 'genre_jazz', 'genre_country',
       'genre_blues', 'genre_folk', 'facebook', 'instagram', 'twitter',
       'youtube'],
      dtype='object')

In [215]:
artists['artist_name']

0                    Shakira
1                    Beyoncé
2             Jennifer Lopez
3                Miley Cyrus
4          Justin Timberlake
5                      Adele
6                Chris Brown
7             Britney Spears
8           Enrique Iglesias
9                    Pitbull
10                    Maluma
11                 Nicky Jam
12                      Pink
13             Avril Lavigne
14             Calvin Harris
15              Romeo Santos
16                      Zayn
17               Alicia Keys
18                Snoop Dogg
19              Harry Styles
20                     Usher
21              Ricky Martin
22              Mariah Carey
23              Prince Royce
24        Christina Aguilera
25                       Sia
26             Martin Garrix
27                Little Mix
28           Camilla Cabello
29                    Thalia
               ...          
100             Diego Torres
101               Kane Brown
102                   Hozier
103           

In [203]:
#Read all social data files in
files = glob.glob('SocialData/*.csv')
files

['SocialData\\facebook_7yrs.csv',
 'SocialData\\instagram_7yrs_cleaned.csv',
 'SocialData\\twitter_features.csv',
 'SocialData\\youtube_to_combine.csv']

In [204]:
df_list = [pd.read_csv(file) for file in files]

C:\Users\oldye\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3214: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [205]:
sum = 0
for d in df_list:
    print("num columns", len(d.columns))
    sum+=d.shape[0]
    print(d.shape)
    print(d.columns)

print("number of rows in 4 dataframes: ", sum)

num columns 22
(297603, 22)
Index(['Unnamed: 0', 'artist_name', 'created', 'description',
       'crowdtangle_score', 'count_of_followers', 'type_link', 'type_photo',
       'type_text', 'type_video', 'description_length', 'created_12am-6am',
       'created_12pm-6pm', 'created_6am-12pm', 'created_6pm-12am',
       'time_since_last_post', 'hashtag_count', 'within_week_release',
       'within_month_release', 'social_engagement_score', 'posts_last_7days',
       'posts_last_30days'],
      dtype='object')
num columns 22
(385381, 22)
Index(['Unnamed: 0', 'artist_name', 'count_of_followers', 'created',
       'description', 'crowdtangle_score', 'description_length',
       'hashtag_count', 'social_engagement_score', 'type_photo', 'type_video',
       'created_12am-6am', 'created_12pm-6pm', 'created_6am-12pm',
       'created_6pm-12am', 'time_since_last_post', 'type_text', 'type_link',
       'posts_last_7days', 'posts_last_30days', 'within_week_release',
       'within_month_release'],
  

In [206]:
#Temporary fixes for youtube
df_list[3].drop(columns='time_of_day', inplace=True)
df_list[3].reset_index(inplace=True)
df_list[3].rename(columns = {'index': 'Unnamed: 0'}, inplace=True)

In [207]:
#Renaming columns that were not aligned
#df_list[0].rename(columns = {'fb_handle_artist_name': 'artist_name'}, inplace=True) #Facebook
df_list[2].rename(columns = {'Twitter_Handle': 'artist_name'}, inplace=True) #Twitter

In [208]:
#Adding Dummy columns to indicate which dataset it is
df_list[0]['Facebook'] = 1
df_list[0]['Instagram'] = 0
df_list[0]['Twitter'] = 0
df_list[0]['Youtube'] = 0

df_list[1]['Facebook'] = 0
df_list[1]['Instagram'] = 1
df_list[1]['Twitter'] = 0
df_list[1]['Youtube'] = 0

df_list[2]['Facebook'] = 0
df_list[2]['Instagram'] = 0
df_list[2]['Twitter'] = 1
df_list[2]['Youtube'] = 0

df_list[3]['Facebook'] = 0
df_list[3]['Instagram'] = 0
df_list[3]['Twitter'] = 0
df_list[3]['Youtube'] = 1

In [209]:
fb = df_list[0].merge(artists, left_on='artist_name', right_on='facebook')
instagram = df_list[1].merge(artists, left_on='artist_name', right_on='instagram')
twitter = df_list[2].merge(artists, left_on='artist_name', right_on='twitter')
yt = df_list[3].merge(artists, left_on='artist_name', right_on='youtube')
socials = [fb, instagram, twitter, yt]

In [210]:
for s in socials:
    print(s.shape)

(297603, 44)
(385381, 44)
(290677, 44)
(20149, 44)


###### Investigation Start

In [174]:
fb_check = df_list[0].merge(artists, how='outer', left_on='artist_name', right_on='facebook')


In [175]:
fb_check.shape

(297603, 44)

In [176]:
#Delta between full outer join and inner join
fb_check.shape[0] - fb.shape[0]

0

In [178]:
fb_diff = pd.concat([fb, fb_check]).drop_duplicates(keep=False)

In [179]:
fb_diff.head()

,Unnamed: 0,artist_name_x,created,description,crowdtangle_score,count_of_followers,type_link,type_photo,type_text,type_video,...,genre_spiritual,genre_electronic,genre_jazz,genre_country,genre_blues,genre_folk,facebook,instagram,twitter,youtube


In [180]:
#Differences match
fb_diff.shape

(0, 44)

In [168]:
fb_diff2 = fb_diff[['artist_name_x', 'facebook', 'instagram', 'twitter', 'youtube']]

In [169]:
#NOTE: THIS is what was missing originally.  Will be blank if re-run
fb_diff2.groupby('artist_name_x').min()

,facebook,instagram,twitter,youtube
artist_name_x,,,,
AliceCooper,NaN,NaN,NaN,NaN
PearlJam,NaN,NaN,NaN,NaN
RickyMartinOfficialPage,NaN,NaN,NaN,NaN
Scorpions,NaN,NaN,NaN,NaN
SiaMusic,NaN,NaN,NaN,NaN
WillieNelson,NaN,NaN,NaN,NaN
fifthharmony,NaN,NaN,NaN,NaN
mirandalambert,NaN,NaN,NaN,NaN
swedishhousemafia,NaN,NaN,NaN,NaN


##### Instagram Check

In [191]:
in_check = df_list[1].merge(artists, how='outer', left_on='artist_name', right_on='instagram')
print(in_check.shape)
#Delta between full outer join and inner join
print(in_check.shape[0] - instagram.shape[0])

in_diff = pd.concat([instagram, in_check]).drop_duplicates(keep=False)

(385381, 44)
0


In [186]:
in_diff.head()

,Unnamed: 0,artist_name_x,count_of_followers,created,description,crowdtangle_score,description_length,hashtag_count,social_engagement_score,type_photo,...,genre_spiritual,genre_electronic,genre_jazz,genre_country,genre_blues,genre_folk,facebook,instagram,twitter,youtube
54968,177844.0,cncomusic,NaN,2015-12-13 23:17:04,¡Bienvenidos a nuestra cuenta oficial! Double ...,-3.2,107.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54969,177851.0,cncomusic,NaN,2015-12-14 00:01:00,¡Gracias @ricky_martin @laurapausini y @alejan...,-4.24,126.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54970,177862.0,cncomusic,NaN,2015-12-14 01:28:18,We made it! 🎤🎼🎉🎊 #CNCO #DevuelvemeMiCorazon,-4.57,43.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54971,177886.0,cncomusic,NaN,2015-12-14 10:05:26,¡Estamos en @despiertaamericatv no te lo pierd...,-4.91,76.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54972,177888.0,cncomusic,NaN,2015-12-14 10:23:17,Gracias @despiertaamericatv por su apoyo🎤 #CNC...,-5.42,68.0,2.0,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [192]:
#Differences match
in_diff.shape

(0, 44)

In [187]:
in_diff2 = in_diff[['artist_name_x', 'facebook', 'instagram', 'twitter', 'youtube']]
in_diff2.groupby('artist_name_x').min()

,facebook,instagram,twitter,youtube
artist_name_x,,,,
cncomusic,NaN,NaN,NaN,NaN
ptxofficial,NaN,NaN,NaN,NaN
swedishhousemafia,NaN,NaN,NaN,NaN
threedaysgraceofficial,NaN,NaN,NaN,NaN


###### investigation stop

In [211]:
df = pd.concat(socials)
df.drop('Unnamed: 0',axis=1,inplace=True)

C:\Users\oldye\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [212]:
df.sample(10)

,Facebook,Instagram,Twitter,Youtube,age,artist_name_x,artist_name_y,count_of_followers,created,created_12am-6am,...,time_since_last_post,twitter,type_link,type_photo,type_text,type_video,within_month_release,within_week_release,years_active,youtube
57976,1,0,0,0,46,Maxwell,Maxwell,NaN,2/17/13 10:32,0,...,0 days 00:05:04.000000000,_MAXWELL_,0,0,1,0,0,0,28,maxwell
86739,0,1,0,0,43,djkhaled,DJ Khaled,2552818.0,2016-01-03 19:04:22,0,...,0 days 01:10:57.000000000,djkhaled,0,0,0,1,0,0,21,DJ Khaled
15470,0,1,0,0,30,asaprocky,ASAP Rocky,5359764.0,2017-02-24 12:37:27,0,...,11 days 13:42:50.000000000,asvpxrocky,0,1,0,0,0,0,12,ASAPROCKYUPTOWN
118137,0,1,0,0,23,fifthharmony,Fifth Harmony,1275901.0,2015-05-14 15:48:09,0,...,2 days 01:28:37.000000000,FifthHarmony,0,1,0,0,0,0,6,Fifth Harmony
120834,1,0,0,0,47,Thalia,Thalia,16243610.0,5/6/14 0:22,1,...,0 days 00:00:00.000000000,thalia,0,1,0,0,0,0,38,Thalia
255743,0,1,0,0,53,pearljam,Pearl Jam,237972.0,2014-12-24 14:35:05,0,...,1 days 00:43:51.000000000,PearlJam,0,1,0,0,0,0,29,Pearl Jam
235948,0,0,1,0,61,montanertwiter,Ricardo Montaner,7334468.0,2016-02-04 20:35:05,0,...,0 days 00:00:23.000000000,montanertwiter,1,0,0,0,0,0,43,Ricardo Montaner
250404,0,0,1,0,26,russdiemon,Russ,84586.0,2016-08-23 03:53:14,1,...,0 days 00:00:21.000000000,russdiemon,0,0,1,0,0,0,12,Russ
79977,0,0,1,0,50,MarcAnthony,Marc Anthony,9683805.0,2017-06-04 23:01:01,0,...,0 days 08:00:20.000000000,MarcAnthony,0,1,0,0,0,0,31,Marc Anthony
271072,0,1,0,0,43,richforever,Rick Ross,NaN,2012-09-10 18:59:59,0,...,0 days 06:58:00.000000000,RickRoss,0,1,0,0,0,0,19,Rick Ross


In [213]:
print(df.shape)
print("number of rows lost: ", sum - df.shape[0], ", ", (sum - df.shape[0])/sum)

(993810, 43)
number of rows lost:  0 ,  0.0


In [214]:
df.head(3).T

,0,1,2
Facebook,1,1,1
Instagram,0,0,0
Twitter,0,0,0
Youtube,0,0,0
age,26,26,26
artist_name_x,21Savage,21Savage,21Savage
artist_name_y,21 Savage,21 Savage,21 Savage
count_of_followers,NaN,NaN,NaN
created,5/18/15 19:24,5/25/15 16:00,5/25/15 22:26
created_12am-6am,0,0,0


In [196]:
df.to_csv('Combined_Social_Data.csv')